In [2]:
from http import client
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType


TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=TRACKING_URI)

In [3]:
client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='NYC-Taxi-Experiment', tags={}>]

In [4]:
client.create_experiment(name = "test-experiment")

'2'

In [21]:
runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.RMSE <6.8",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

for run in runs:
    print("######################################################################################")
    print(f"Run ID: {run.info.run_id}, RMSE: {run.data.metrics['RMSE']}")

######################################################################################
Run ID: 2d5148ea517c49cfaf209c704614f4b9, RMSE: 6.742303328497426
######################################################################################
Run ID: 8a3bef86375a482ba4f77ddaac2df6e0, RMSE: 6.303762622835327


In [3]:
import mlflow

mlflow.set_tracking_uri(TRACKING_URI)

In [23]:
run_id = "8a3bef86375a482ba4f77ddaac2df6e0"
model_uri = f"runs:/{run_id}/model"

mlflow.register_model(
    model_uri=model_uri,
    name="nyc-registered-model")

Registered model 'nyc-registered-model' already exists. Creating a new version of this model...
2022/10/13 09:19:24 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-registered-model, version 3
Created version '3' of model 'nyc-registered-model'.


<ModelVersion: creation_timestamp=1665632964071, current_stage='None', description=None, last_updated_timestamp=1665632964071, name='nyc-registered-model', run_id='8a3bef86375a482ba4f77ddaac2df6e0', run_link=None, source='./mlruns/1/8a3bef86375a482ba4f77ddaac2df6e0/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [24]:
# Transition a model

client.list_registered_models()

[<RegisteredModel: creation_timestamp=1665631210554, description='NYC taxi trip duration prediction', last_updated_timestamp=1665632964071, latest_versions=[<ModelVersion: creation_timestamp=1665631223938, current_stage='Staging', description='', last_updated_timestamp=1665631413036, name='nyc-registered-model', run_id='2d5148ea517c49cfaf209c704614f4b9', run_link='', source='./mlruns/1/2d5148ea517c49cfaf209c704614f4b9/artifacts/model', status='READY', status_message=None, tags={'model': 'gradboostregressor'}, user_id=None, version=2>,
  <ModelVersion: creation_timestamp=1665632964071, current_stage='None', description=None, last_updated_timestamp=1665632964071, name='nyc-registered-model', run_id='8a3bef86375a482ba4f77ddaac2df6e0', run_link=None, source='./mlruns/1/8a3bef86375a482ba4f77ddaac2df6e0/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>], name='nyc-registered-model', tags={}>]

In [31]:
model_name = "nyc-registered-model"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"Stage: {version.current_stage}, Version: {version.version}")

Stage: Production, Version: 1
Stage: Staging, Version: 3


In [34]:
model_version=2
new_stage = "Staging"

client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1665631223938, current_stage='Staging', description='', last_updated_timestamp=1665633604639, name='nyc-registered-model', run_id='2d5148ea517c49cfaf209c704614f4b9', run_link='', source='./mlruns/1/2d5148ea517c49cfaf209c704614f4b9/artifacts/model', status='READY', status_message=None, tags={'model': 'gradboostregressor'}, user_id=None, version=2>

In [35]:
from datetime import datetime


client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"Model Version {model_version} is moved to {new_stage} on {datetime.today().date()}"
)

<ModelVersion: creation_timestamp=1665631223938, current_stage='Staging', description='Model Version 2 is moved to Staging on 2022-10-13', last_updated_timestamp=1665633772621, name='nyc-registered-model', run_id='2d5148ea517c49cfaf209c704614f4b9', run_link='', source='./mlruns/1/2d5148ea517c49cfaf209c704614f4b9/artifacts/model', status='READY', status_message=None, tags={'model': 'gradboostregressor'}, user_id=None, version=2>

In [4]:
from sklearn.metrics import mean_squared_error
import pandas as pd

In [5]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [6]:
df = read_dataframe("./data/green_tripdata_2021-03.parquet")

In [7]:
client.download_artifacts(run_id="d9a4c6e3bfa841a79b0f9b1046c92d40", path='preprocessor', dst_path='.')

'c:\\Users\\soumy\\source\\repos\\mlops-zoomcamp\\02-experiment-tracking\\preprocessor'

In [8]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [9]:
X_test = preprocess(df, dv)

In [10]:
target = "duration"
y_test = df[target].values

In [12]:
model_name = "nyc-registered-model"
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

[13:17:46] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: total: 2.44 s
Wall time: 308 ms


{'rmse': 6.255787549981744}

In [13]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

CPU times: total: 31.2 ms
Wall time: 158 ms


{'rmse': 6.659623830022514}

In [14]:
client.transition_model_version_stage(
    name=model_name,
    version=2,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: creation_timestamp=1665631223938, current_stage='Production', description='Model Version 2 is moved to Staging on 2022-10-13', last_updated_timestamp=1665647382782, name='nyc-registered-model', run_id='2d5148ea517c49cfaf209c704614f4b9', run_link='', source='./mlruns/1/2d5148ea517c49cfaf209c704614f4b9/artifacts/model', status='READY', status_message=None, tags={'model': 'gradboostregressor'}, user_id=None, version=2>